In [1]:
import spacy
import pandas as pd
import numpy as np
import time
from statistics import mean 
from multiprocessing import Pool
import pickle
from spacy.tokens import DocBin

pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

nlp = spacy.load("pt_core_news_sm")

In [2]:
df = pd.read_csv("MERCADO_FINAL.csv")
port = pd.read_csv("estaticos_portfolio1.csv")
df_port = df[df['id'].isin(port['id'])].reset_index(drop=True)

In [36]:
len(df)

462298

In [39]:
df

,id,0
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,sociedade empresaria limitada entidades empres...
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,empresario individual outros servicos alojamen...
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,empresario individual outros transporte armaze...
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,empresario individual outros servicos diversos...
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,empresario individual outros servicos profissi...
...,...,...
462293,469ee5fc984d0bb60aa2f40cda4d003d7c9b963d149ed3...,empresario individual outros comercio reparaca...
462294,656f496e7e2fad708f0022d8ad97d2cb9344e839a5f3cb...,empresario individual outros servicos administ...
462295,9b933a5cf2149d1338f00fb459d34a16b9b0f2e8d3e8e1...,associacao privada entidades fins lucrativos s...
462296,84db0f17467cf4f02439e01ade57150b6ac59d9dae53ed...,sociedade empresaria limitada entidades empres...


In [37]:
len(df_dupl)

6832

In [38]:
df_dupl

,id,0,Docs
0,6c45aabb849a760a3a7107c7a61262642f683feb8912a4...,cooperativa entidades empresariais bens consum...,"(cooperativa, entidades, empresariais, bens, c..."
1,764bb8648b96fd6459fa2f2237c22f3d6b8c129a814e33...,consorcio sociedades entidades empresariais in...,"(consorcio, sociedades, entidades, empresariai..."
2,a81c8e1d49c50f9ee7e5e7b25cd2223eba13ffcff402fe...,sociedade empresaria limitada entidades empres...,"(sociedade, empresaria, limitada, entidades, e..."
3,128ad43b6d43ab63679620906fc4bcb733f7561513967d...,orgao publico poder executivo municipal admini...,"(orgao, publico, poder, executivo, municipal, ..."
4,a820394924e9d954f40431b995072bd5c4b8b7b1239d59...,empresa individual responsabilidade limitada n...,"(empresa, individual, responsabilidade, limita..."
...,...,...,...
6827,469ee5fc984d0bb60aa2f40cda4d003d7c9b963d149ed3...,empresario individual outros comercio reparaca...,"(empresario, individual, outros, comercio, rep..."
6828,656f496e7e2fad708f0022d8ad97d2cb9344e839a5f3cb...,empresario individual outros servicos administ...,"(empresario, individual, outros, servicos, adm..."
6829,9b933a5cf2149d1338f00fb459d34a16b9b0f2e8d3e8e1...,associacao privada entidades fins lucrativos s...,"(associacao, privada, entidades, fins, lucrati..."
6830,84db0f17467cf4f02439e01ade57150b6ac59d9dae53ed...,sociedade empresaria limitada entidades empres...,"(sociedade, empresaria, limitada, entidades, e..."


In [3]:
df_dupl = df.drop_duplicates(subset='0', keep="last").reset_index(drop=True)
df_port_dupl = df_port.drop_duplicates(subset='0', keep="last").reset_index(drop=True)
dici_dupl = df['0'].value_counts().to_dict()
dici_port = df_port['0'].value_counts().to_dict()

In [4]:
def tokeniza(df):
    df["Docs"] = df["0"].apply(lambda x: nlp(x)).to_list()
    return df

In [5]:
def parallelize_dataframe(df, func, n_cores=12):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.terminate()
    pool.join()
    return df
#     lista_final=[]
#     for i in lista:
#         lista_final.append(i)

#     return lista_final

In [6]:
start_time = time.time()
df_dupl = parallelize_dataframe(df_dupl,tokeniza)
print("--- %s seconds ---" % (time.time() - start_time))

--- 45.005906105041504 seconds ---


In [7]:
start_time = time.time()
df_port_dupl = parallelize_dataframe(df_port_dupl,tokeniza)
print("--- %s seconds ---" % (time.time() - start_time))

--- 15.874993085861206 seconds ---


In [8]:
def compara(df):
    df_vazio=pd.DataFrame(columns=["id","similarity"])
    #itera mercado
    for i in range(len(df)):
        doc_mercado = df["Docs"].iloc[i]
        similarity_sum=0
        #iteraportfolio
        for j in range(len(df_port_dupl)):
            doc_portfolio = df_port_dupl["Docs"][j]
            similarity_sum+=(doc_mercado.similarity(doc_portfolio)*dici_port[df_port_dupl["0"][j]])
            semelhanca_media = similarity_sum/len(df_port)
        df_vazio = df_vazio.append({'id':df['id'].iloc[i],'similarity':semelhanca_media},ignore_index=True)
    #df["Similarity"]=df.apply(lambda row: (doc_mercado.similarity(row['Docs']) * dici_port[row["0"]]),axis=1)
    return df_vazio

In [9]:
def parallelize_compare(df, func, n_cores=12):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split)).reset_index(drop=True)
    pool.close()
    pool.terminate()
    pool.join()
    return df

In [10]:
%%capture
start_time = time.time()
#itera mercado
df_vazio = parallelize_compare(df_dupl,compara)
tempo_total = (time.time() - start_time)
print("--- %s seconds ---" % (time.time() - start_time))

In [24]:
print("--- %s seconds ---" ,tempo_total)

--- %s seconds --- 2038.9874329566956


In [12]:
df_vazio

,id,similarity
0,6c45aabb849a760a3a7107c7a61262642f683feb8912a4...,0.839776
1,764bb8648b96fd6459fa2f2237c22f3d6b8c129a814e33...,0.892028
2,a81c8e1d49c50f9ee7e5e7b25cd2223eba13ffcff402fe...,0.846356
3,128ad43b6d43ab63679620906fc4bcb733f7561513967d...,0.878410
4,a820394924e9d954f40431b995072bd5c4b8b7b1239d59...,0.815033
...,...,...
6827,469ee5fc984d0bb60aa2f40cda4d003d7c9b963d149ed3...,0.924140
6828,656f496e7e2fad708f0022d8ad97d2cb9344e839a5f3cb...,0.868174
6829,9b933a5cf2149d1338f00fb459d34a16b9b0f2e8d3e8e1...,0.857196
6830,84db0f17467cf4f02439e01ade57150b6ac59d9dae53ed...,0.922097


In [13]:
# start_time = time.time()
# df_vazio=pd.DataFrame(columns=["id","similarity"])
# #itera mercado
# for i in range(len(df_dupl)):
#     doc_mercado = df_dupl["Docs"][i]
#     semelhanca_com_port=[]
#     #itera portfolio
#     for j in range(len(df_port_dupl)):
#         doc_portfolio = df_port_dupl["Docs"][j]
#         semelhanca_com_port.append(doc_mercado.similarity(doc_portfolio) * dici_port[df_port_dupl["0"][j]])
#     semelhanca_media = sum(semelhanca_com_port)/len(df_port)
#     df_vazio = df_vazio.append({'id':df_dupl['id'][i],'similarity':semelhanca_media},ignore_index=True)
#     print(i)
# tempo_total = (time.time() - start_time)
# print("--- %s seconds ---" % (time.time() - start_time))

In [14]:
# tempo_total

In [34]:
df_vazio.sort_values(by="similarity",ascending=False,ignore_index=True).head(530)

,id,similarity
0,44b1d031f0bff84d246181e97a7908a317cdfb8c0e096b...,0.968894
1,ba57e8f8416d1ca3cb2bd92e7e69c5bd6eb2807770ae0d...,0.968748
2,41cbf10a3e371d42988367df625f3faa1b1aeee74715a0...,0.968418
3,1b84118b02038a999c5f10b08ad97b7773b4221a501d68...,0.968317
4,25fc6c2f86ea82180b19935893969ca5943af053cf4fa6...,0.967975
...,...,...
525,ac9f147230570bb1611128fe56791f449ec3d3008d34eb...,0.900497
526,8017c3611dba9a74c34daa76e60247ca22d512b80c0730...,0.900476
527,ce73c913ea249ee8ad4f5fe39a6221fe8749b5f9408b91...,0.900327
528,41e503ed0eefea89a620d4f86dbd292d755187fdeb1eeb...,0.900227


In [40]:
df_vazio.sort_values("similarity",ascending=False).reset_index(drop=True)["id"][0]

'44b1d031f0bff84d246181e97a7908a317cdfb8c0e096b937e08e5729c376d6b'

In [41]:
df[df["id"]=='44b1d031f0bff84d246181e97a7908a317cdfb8c0e096b937e08e5729c376d6b'].reset_index(drop=True)["0"][0]

'empresario individual outros comercio varejista comercio comercio varejista comercio reparacao veiculos automotores motocicletas norte maranhense'

In [51]:
for token in df_dupl["Docs"][0]:
    print(token.text, token.lemma_, token.pos_, token.dep_, token.ent_type)

cooperativa cooperativo VERB ROOT 0
entidades entidade NOUN obj 0
empresariais empresarial ADJ amod 0
bens bem ADJ nsubj 0
consumo consumir ADJ amod 0
industria industriar ADJ ROOT 0
fabricacao fabricacao VERB advcl 0
produtos produto NOUN obj 0
diversos diverso ADJ det 0
industrias industriar ADJ nsubj 0
transformacao transformacao VERB ROOT 0
centro centrar PROPN obj 0
amazonense amazonense PROPN amod 0


In [18]:
# filename = 'modelo_mercado.sav'

In [19]:
# doc_bin = DocBin(attrs=["POS", "TAG", "ENT_TYPE"], store_user_data=True)
# for doc in df22.Docs:
#     doc_bin.add(doc)
# bytes_data = doc_bin.to_bytes()
# pickle.dump(bytes_data, open( filename, "wb" ))

In [20]:
# loaded_model = pickle.load(open(filename, 'rb'))
# doc_bin1 = DocBin(attrs=["POS", "TAG", "ENT_TYPE"]).from_bytes(bytes_data)
# docs = list(doc_bin.get_docs(nlp.vocab))

In [21]:
# for token in docs[0]:
#     print(token.text, token.lemma_, token.pos_, token.dep_)

In [22]:
# start_time = time.time()

# #itera mercado
# l_mercado=[]
# for i in range(10):
#     l_mercado.append(nlp(df["0"][i]))

# print("--- %s seconds ---" % (time.time() - start_time))

In [23]:
# #itera portfolio
# start_time = time.time()

# l_portfolio=[]
# for i in range(len(df_port_dupl)):
#     l_portfolio.append(nlp(df_port_dupl["0"][i]))
        
# print("--- %s seconds ---" % (time.time() - start_time))